<a href="https://colab.research.google.com/github/maciejskorski/distorted_clustering/blob/main/Clustering_DistortedMetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

This notebook demonstrates how to cluster under distorted metrics, like the huber-like loss functions. Such metrics have been recently shown to be statistically more robust and accuracte for various machine learning tasks (see the paper by Google https://arxiv.org/abs/1701.03077)

Unfortunately, fitting such models is harder than for KMeans, and is not supported in off-the-shell software, such as scikitlearn. 

This notebook offers a Tensorflow implementation along with examples on real-world data.

# Model

We use the Expectation-Maximization algorithm (EM). The idea is essentially as in the case of KMeans; however KMeans works under the euclidean distance and allows for solving the M step analytically. In our case we resort to generic (gradient-based) optimization.


## Tensorflow 1.x

In [ ]:
### Tensorflow Model

%tensorflow_version 1.x
import tensorflow as tf

def logp_huber(scale=1):

  def logp_huber(x):
    return -scale**2*((1+tf.square(x/scale))**0.5-1)
  
  return logp_huber

def logp_euclidean():

  def logp_euclidean(x):
    return -tf.square(x)

  return logp_euclidean

def build_cluster_graph(data_shape,n_clusters,logp_func=logp_euclidean,init=None):

  ## model variables 

  n_rows,n_features = data_shape
  X_t = tf.placeholder(tf.float32,shape=[n_rows,n_features]) # data
  y_logp = tf.Variable(tf.log(tf.ones(shape=[n_rows,n_clusters])/n_features),trainable=False) # class predictions, updated by Bayes rules
  sigmas = tf.ones(shape=[n_features]) # standard deviations: here fixed
  if init is None:
    init = tf.random.normal(shape=[n_clusters,n_features])
  centers = tf.Variable(init,trainable=True) # centers: trainable

  ## data probability given class predictions and model params: log Pr[x|y,model]

  diff = tf.expand_dims(X_t,axis=1) - tf.expand_dims(centers,0) # [n_rows,n_clusters,n_features]
  X_logp = tf.reduce_sum(logp_func(diff),axis=2) # [n_rows,n_clusters]

  ## optimize params given class predictions: log Pr[x|y]

  loglike = tf.reduce_logsumexp(y_logp + X_logp,axis=1) # [n_rows]
  neg_loglike = -tf.reduce_mean(loglike) # []
  optimizer = tf.train.AdamOptimizer()
  mstep = optimizer.minimize(neg_loglike)

  ## optimize class predictions given params via Bayes: Pr[y|x] := Pr[x|y]Pr[y] / Pr[x] 

  y_logp_new = X_logp+y_logp - tf.reduce_logsumexp(X_logp+y_logp,axis=1,keepdims=True)
  estep = tf.assign(y_logp,y_logp_new)

  init_op = tf.global_variables_initializer()

  return X_t,y_logp,mstep,estep,neg_loglike

TensorFlow 1.x selected.


# Tests


We demonstrate improvements when using Huber loss for clustering, 
on some standard clustering benchmarks.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score,adjusted_mutual_info_score

## User Knowledge Dataset


In [ ]:
## download and prepare data

!mkdir /content/user_knowledge
!curl -o /content/user_knowledge/data.xls 'https://archive.ics.uci.edu/ml/machine-learning-databases/00257/Data_User_Modeling_Dataset_Hamdi%20Tolga%20KAHRAMAN.xls'
X=pd.read_excel('user_knowledge/data.xls',sheet_name=1)
X=X[X.columns[:6]]
print(X)
X=X.to_numpy()
X,y=X[:,:-1],X[:,-1]
X=X.astype('float')
encode_dict={v:k for k,v in enumerate(np.unique(y))}
y=np.array(list(encode_dict[i] for i in y))
X=(X-X.mean(0))/X.std(0)

mkdir: cannot create directory ‘/content/user_knowledge’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57856  100 57856    0     0   106k      0 --:--:-- --:--:-- --:--:--  105k
      STG   SCG   STR   LPR   PEG       UNS
0    0.00  0.00  0.00  0.00  0.00  very_low
1    0.08  0.08  0.10  0.24  0.90      High
2    0.06  0.06  0.05  0.25  0.33       Low
3    0.10  0.10  0.15  0.65  0.30    Middle
4    0.08  0.08  0.08  0.98  0.24       Low
..    ...   ...   ...   ...   ...       ...
253  0.61  0.78  0.69  0.92  0.58      High
254  0.78  0.61  0.71  0.19  0.60    Middle
255  0.54  0.82  0.71  0.29  0.77      High
256  0.50  0.75  0.81  0.61  0.26    Middle
257  0.66  0.90  0.76  0.87  0.74      High

[258 rows x 6 columns]


In [ ]:
## cluster by KMeans

np.random.seed(1234)
y_pred = KMeans(n_clusters=len(np.unique(y)),n_init=10).fit_predict(X)
print(adjusted_rand_score(y,y_pred),adjusted_mutual_info_score(y,y_pred))

0.16027687114838174 0.20584592211410666


In [ ]:
## cluster by Huber

outs = []

n_clusters=len(np.unique(y))
n_rows,n_features = X.shape

for _ in range(10):
  
  for logp_func in [logp_huber(0.25)]:

    tf.reset_default_graph()
    init = np.random.normal(size=(n_clusters,n_features)).astype(np.float32)
    X_t,y_logp,mstep,estep,loglike = build_cluster_graph((n_rows,n_features),n_clusters,logp_func=logp_func,init=init)
    init_op = tf.global_variables_initializer()

    ## train
    with tf.Session() as sess:
      feed_dict = {X_t:X}
      sess.run(init_op)
      # do iterations
      val = 0
      for i in range(50):
        # m step (many times to get close to maximum likelihood)
        val_m = 0
        for _ in range(1000):
          _,val_m_new = sess.run([mstep,loglike],feed_dict=feed_dict)
          if abs(val_m_new-val_m) < 1e-5:
            break
          else:
            val_m = val_m_new
        # e step (once as it is analytic)
        sess.run(estep,feed_dict=feed_dict)

      y_pred = sess.run(y_logp,feed_dict)

    score1 = adjusted_rand_score(y,y_pred.argmax(1))
    score2 = adjusted_mutual_info_score(y,y_pred.argmax(1))
    outs.append((logp_func.__name__,val_m,score1,score2))
    
outs = pd.DataFrame(outs,columns=['method','loglike','ARI','AMI'])
print(outs.sort_values('loglike')[:2].mean())
outs

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
loglike    0.555272
ARI        0.189464
AMI        0.212478
dtype: float64


,method,loglike,ARI,AMI
0,logp_huber,0.571304,0.159812,0.212525
1,logp_huber,0.559494,0.140852,0.180000
2,logp_huber,0.557813,0.203783,0.292465
3,logp_huber,0.553296,0.174851,0.195897
4,logp_huber,0.560340,0.171611,0.237812
5,logp_huber,0.569612,0.113819,0.136037
6,logp_huber,0.579853,0.152204,0.183827
7,logp_huber,0.557249,0.204077,0.229058
8,logp_huber,0.559522,0.161576,0.220254
9,logp_huber,0.561033,0.185360,0.226004


## Mice Protein Nuclear Dataset

In [ ]:
## download and prepare data

!mkdir /content/mice
!curl -o /content/mice/data.xls 'https://archive.ics.uci.edu/ml/machine-learning-databases/00342/Data_Cortex_Nuclear.xls'

X=pd.read_excel('mice/data.xls')
# do one-hot-encoding for categorical data
print(X.head())
for c in X.columns[X.dtypes=='object']:
  encode_dict = {v:k for k,v in enumerate(X[c].unique())}
  X[c] = X[c].apply(encode_dict.get)
# fill missing data
X = X.fillna(X.median())
X = X.to_numpy()
X,y = X[:,:-1],X[:,-1]
X = (X-X.mean(0))/X.std(0)

mkdir: cannot create directory ‘/content/mice’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1590k  100 1590k    0     0  2178k      0 --:--:-- --:--:-- --:--:-- 2175k
  MouseID  DYRK1A_N   ITSN1_N    BDNF_N  ...  Genotype  Treatment  Behavior   class
0   309_1  0.503644  0.747193  0.430175  ...   Control  Memantine       C/S  c-CS-m
1   309_2  0.514617  0.689064  0.411770  ...   Control  Memantine       C/S  c-CS-m
2   309_3  0.509183  0.730247  0.418309  ...   Control  Memantine       C/S  c-CS-m
3   309_4  0.442107  0.617076  0.358626  ...   Control  Memantine       C/S  c-CS-m
4   309_5  0.434940  0.617430  0.358802  ...   Control  Memantine       C/S  c-CS-m

[5 rows x 82 columns]


In [ ]:
## cluster by KMeans

np.random.seed(1234)
y_pred = KMeans(n_clusters=len(np.unique(y)),n_init=10).fit_predict(X)
print(adjusted_rand_score(y,y_pred),adjusted_rand_score(y,y_pred))

0.1926732829622158 0.1926732829622158


In [ ]:
## cluster by Huber

outs = []

n_clusters=len(np.unique(y))
n_rows,n_features = X.shape

for _ in range(5):
  
  for logp_func in [logp_huber(0.25)]:

    tf.reset_default_graph()
    init = np.random.normal(size=(n_clusters,n_features)).astype(np.float32)
    X_t,y_logp,mstep,estep,loglike = build_cluster_graph((n_rows,n_features),n_clusters,logp_func=logp_func,init=init)
    init_op = tf.global_variables_initializer()

    ## train
    with tf.Session() as sess:
      feed_dict = {X_t:X}
      sess.run(init_op)
      # do iterations
      val = 0
      for i in range(50):
        # m step (many times to get close to maximum likelihood)
        val_m = 0
        for _ in range(1000):
          _,val_m_new = sess.run([mstep,loglike],feed_dict=feed_dict)
          if abs(val_m_new-val_m) < 1e-5:
            break
          else:
            val_m = val_m_new
        # e step (once as it is analytic)
        sess.run(estep,feed_dict=feed_dict)

      y_pred = sess.run(y_logp,feed_dict)

    score = adjusted_rand_score(y,y_pred.argmax(1))
    outs.append((logp_func.__name__,val_m,score))
    
outs = pd.DataFrame(outs,columns=['method','loglike','ARI'])
print(outs.sort_values('loglike')[:2].mean())
outs

loglike    8.109578
ARI        0.206287
dtype: float64


,method,loglike,ARI
0,logp_huber,8.159964,0.150249
1,logp_huber,8.197568,0.228981
2,logp_huber,8.098990,0.193778
3,logp_huber,8.133040,0.223915
4,logp_huber,8.120166,0.218797


## Alcohol Dataset

In [ ]:
# download and prepare data

!mkdir /content/alkoholqcm
!curl -o /content/alkoholqcm/data.zip 'https://archive.ics.uci.edu/ml/machine-learning-databases/00496/QCM%20Sensor%20Alcohol%20Dataset.zip'
!unzip -d /content/alkoholqcm  /content/alkoholqcm/data.zip

import os
out = []
for fname in os.listdir('alkoholqcm/QCM Sensor Alcohol Dataset'):
  out.append(pd.read_csv('alkoholqcm/QCM Sensor Alcohol Dataset/'+fname,sep=';',header=None,skiprows=1))
X=pd.concat(out)
print(X.head())
X = X.to_numpy()
X,y = X[:,:-5],X[:,-5:]
y = y.argmax(1)
X = (X-X.mean(0))/X.std(0)

mkdir: cannot create directory ‘/content/alkoholqcm’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5704  100  5704    0     0  20444      0 --:--:-- --:--:-- --:--:-- 20444
Archive:  /content/alkoholqcm/data.zip
replace /content/alkoholqcm/QCM Sensor Alcohol Dataset/QCM10.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/alkoholqcm/QCM Sensor Alcohol Dataset/QCM10.csv  
  inflating: /content/alkoholqcm/QCM Sensor Alcohol Dataset/QCM12.csv  
  inflating: /content/alkoholqcm/QCM Sensor Alcohol Dataset/QCM3.csv  
  inflating: /content/alkoholqcm/QCM Sensor Alcohol Dataset/QCM6.csv  
  inflating: /content/alkoholqcm/QCM Sensor Alcohol Dataset/QCM7.csv  
      0      1      2      3      4      5   ...      9   10  11  12  13  14
0 -11.82 -13.29 -19.32 -26.28 -38.14 -50.09  ... -104.66   1   0   0   0   0
1 -11.54 -14.18 -25.35 -32.75 -48.77 -60.

In [ ]:
## cluster by KMeans

np.random.seed(1234)
y_pred = KMeans(n_clusters=len(np.unique(y)),n_init=10).fit_predict(X)
print(adjusted_rand_score(y,y_pred),adjusted_mutual_info_score(y,y_pred))

0.22795101368735207 0.297064834957504


In [ ]:
## cluster by Huber

outs = []

n_clusters=len(np.unique(y))
n_rows,n_features = X.shape

for _ in range(5):
  
  for logp_func in [logp_huber(0.25)]:

    tf.reset_default_graph()
    init = np.random.normal(size=(n_clusters,n_features)).astype(np.float32)
    X_t,y_logp,mstep,estep,loglike = build_cluster_graph((n_rows,n_features),n_clusters,logp_func=logp_func,init=init)
    init_op = tf.global_variables_initializer()

    ## train
    with tf.Session() as sess:
      feed_dict = {X_t:X}
      sess.run(init_op)
      # do iterations
      val = 0
      for i in range(50):
        # m step (many times to get close to maximum likelihood)
        val_m = 0
        for _ in range(1000):
          _,val_m_new = sess.run([mstep,loglike],feed_dict=feed_dict)
          if abs(val_m_new-val_m) < 1e-5:
            break
          else:
            val_m = val_m_new
        # e step (once as it is analytic)
        sess.run(estep,feed_dict=feed_dict)

      y_pred = sess.run(y_logp,feed_dict)

    score1 = adjusted_rand_score(y,y_pred.argmax(1))
    score2 = adjusted_mutual_info_score(y,y_pred.argmax(1))
    outs.append((logp_func.__name__,val_m,score1,score2))
    
outs = pd.DataFrame(outs,columns=['method','loglike','ARI','AMI'])
print(outs.sort_values('loglike')[:2].mean())
outs

loglike    0.294912
ARI        0.260209
AMI        0.329549
dtype: float64


,method,loglike,ARI,AMI
0,logp_huber,0.297712,0.249461,0.317672
1,logp_huber,0.296295,0.267094,0.335799
2,logp_huber,0.296622,0.267094,0.335799
3,logp_huber,0.306297,0.245432,0.330365
4,logp_huber,0.293528,0.253324,0.323299


# Experimental: Tensorflow 2.x

In [ ]:
### Tensorflow Model

%tensorflow_version 2.x
import tensorflow as tf

def logp_euclidean(x):
  ''' probability based on euclidean distance '''
  return -tf.square(x)

def logp_huber(scale):
  ''' probability based on huber distance '''

  def logp_huber(x):
    return -scale**2*((1+tf.square(x/scale))**0.5-1)

  return logp_huber

def train(X,n_clusters=3,logp_func=logp_euclidean,init=None):

  ## read dataset description

  n_rows,n_features = X.shape
  n_clusters = 3

  ## model variables

  # cluster probabilities
  y_logp = tf.Variable(tf.math.log(tf.ones(shape=[n_rows,n_clusters])/n_features))
  # cluster variances, here fixed (simpler math)
  sigmas = tf.ones(shape=[n_features]) # standard deviations: here fixed
  # clusters centers
  if init is None:
    init = tf.random.normal(shape=[n_clusters,n_features])
  centers = tf.Variable(init) # [n_clusters,n_features]

  ## probability calculations, given cluster probabilities

  def X_y_logp():
    offset = tf.expand_dims(X,axis=1)  - tf.expand_dims(centers,0) # [n_rows,n_clusters,n_features]
    return tf.reduce_sum(logp_func(offset),axis=2) # [n_rows,n_clusters]

  def X_logp():
    X_logp = X_y_logp() # [n_rows,n_clusters]
    return tf.reduce_logsumexp(X_logp + y_logp,axis=1,keepdims=True) # [n_rows]

  def logp():
    return tf.reduce_mean(X_logp()) # []

  ## update class probs (e-step)

  @tf.function
  def estep():
    y_logp.assign(X_y_logp() + y_logp - X_logp())

  ## update cluster locations (m-step)

  optimizer = tf.optimizers.Adam()

  @tf.function
  def mstep():
    optimizer.minimize(lambda: -logp(),[centers])

  for _ in range(50):
    m = 0
    for _ in range(100):
      mstep()
      m_new = -logp()
      if abs(m_new-m)<1e-4:
        break
      else:
        m = m_new
    estep()

  return y_logp.numpy(),m.numpy()

In [ ]:
import numpy as np
np.random.seed(666)

from sklearn import datasets
from sklearn.metrics.cluster import adjusted_rand_score

iris = datasets.load_wine()
X = iris.data.astype('float32')
y = iris.target.astype('int32')
X = (X-X.mean(0))/X.std(0)

n_features = X.shape[1]
n_clusters = len(np.unique(y))

for scale in [0.01,0.1,0.25,0.5,1,5]:
  print(scale)
  for _ in range(5):
    init = np.random.normal(size=(n_clusters,n_features)).astype(np.float32)
    y_pred,logp = train(X,n_clusters,logp_huber(scale),init)
    print('ari=%s,logp=%s'%(adjusted_rand_score(y_pred.argmax(1),y),logp))

0.01
ari=0.0778678618108011,logp=0.11990287
ari=0.3203694454096623,logp=0.113562495
ari=0.24801772963076452,logp=0.120621316
ari=0.17445954160586385,logp=0.13294195
ari=0.2437744669020708,logp=0.12638737
0.1
ari=0.7537819920705809,logp=0.64651144
ari=0.3755508420147064,logp=0.7114152
ari=0.6644891723287554,logp=0.6498259
ari=0.4278674623767463,logp=0.7699368
ari=0.8162825428066811,logp=0.6710881
0.25
ari=0.8000727134982856,logp=1.2828132
ari=0.8368297906511479,logp=1.3285872
ari=0.689578596531488,logp=1.3171464
ari=0.3238745336131592,logp=1.5650926
ari=0.8637747871659045,logp=1.278671
0.5
ari=0.45577731264146465,logp=2.4726663
ari=0.7002911871348606,logp=1.9996917
ari=0.912130038265503,logp=1.964745
ari=0.6052246062377723,logp=2.0473838
ari=0.7711034095636224,logp=2.0088317
1
ari=0.8313336205700521,logp=2.6896696
ari=0.3698552896570588,logp=3.3406065
ari=0.8791908099398843,logp=2.690258
ari=0.8158755966377977,logp=2.706981
ari=0.8150068438497655,logp=2.6871662
5
ari=0.6862401409895932,

In [ ]:
from sklearn import cluster

adjusted_rand_score(cluster.KMeans(n_clusters).fit_predict(X),y)

0.8974949815093207